This notebook serves to get additional data from the EDH XML/epidoc files. The reason is that some information is missing in the API data.


In [0]:
### REQUIREMENTS
import numpy as np
import math
import pandas as pd

import sys
### we do a lot of requests during the scrapping. Some of them with requests package, some of them with urllib
import requests
from urllib.request import urlopen 
from urllib.parse import quote  
from bs4 import BeautifulSoup
import xml.etree.cElementTree as ET

# to avoid errors, we sometime use time.sleep(N) before retrying a request
import time
# the input data have typically a json structure
import json
import getpass

import datetime as dt
# for simple paralel computing:
from concurrent.futures import ThreadPoolExecutor
### google drive
from google.colab import drive
#import gspread
#from gspread_dataframe import get_as_dataframe, set_with_dataframe

In [0]:
### perhaps we will pull the data directly from GitHub at some point

!pip install PyGithub
from github import Github

In [0]:
# using username and password
github_login = "kasev"
g = Github("kasev", getpass.getpass())

In [0]:
repo = g.get_repo("sdam-au/edh_workflow")


## One file parsing test

In [3]:
url = "https://raw.githubusercontent.com/sdam-au/edh_workflow/master/original_dataset/EDH_dump/HD000001.xml"
r = requests.get(url)
r.text

'<?xml version="1.0" encoding="UTF-8"?><?xml-model href="http://www.stoa.org/epidoc/schema/latest/tei-epidoc.rng" schematypens="http://relaxng.org/ns/structure/1.0"?><TEI xmlns="http://www.tei-c.org/ns/1.0" xml:space="preserve" xml:lang="de" xml:base="ex-epidoctemplate.xml">\n    <teiHeader>\n        <fileDesc>\n            <titleStmt>\n                <title>Grabinschrift auf Tafel</title>\n            </titleStmt>  \n            <publicationStmt>\n                <authority>Epigraphische Datenbank Heidelberg</authority>\n                <idno type="URI">http://edh-www.adw.uni-heidelberg.de/edh/inschrift/HD000001</idno>\n                <idno type="TM">251193</idno><idno type="localID">HD000001</idno>\n                <availability>\n                    <p>© Heidelberg Academy of Sciences and Humanities</p>\n                    <licence target="http://creativecommons.org/licenses/by-sa/4.0/">This file is licensed under the Creative Commons Attribution-ShareAlike 4.0 license.\n        

In [0]:
soup = BeautifulSoup(r.text)
soup

In [0]:
url = "https://raw.githubusercontent.com/sdam-au/edh_workflow/master/original_dataset/EDH_dump/HD000001.xml"
soup = BeautifulSoup(requests.get(url).text)

values = {}
tag_names = []
n = 2
for tag in soup.find_all():
  tag_values = tag.attrs
  tag_values.update({"get_text": tag.get_text()})
  tag_name = str(tag.name)
  value = tag.get_text()
  #if len(soup.find_all(tag_name)) > 1:
  for parent in tag.parents:
    tag_name = str(parent.name) + "/" + tag_name
  tag_name = tag_name.partition("tei/")[2]
  if tag_name in tag_names:
    tag_name = tag_name + "_" + str(n)
    n = n + 1
  else:
    tag_names.append(tag_name)
    n = 2
  values.update({tag_name : tag_values})

In [0]:
values

## Main function - 80,000 xml files